# Crunching ARCO data and saving results on LEAP-Pangeo

In this tutorial we will see how we can lazily load a gigantic analysis-ready cloud-optimized (ARCO) dataset, apply our custom analysis to it, and save the results to LEAP cloud storage.

## Our Dataset: ARCO ERA5

The dataset we are using is a prime example where the work by some dedicated folks to ingest and transform a dataset to an ARCO zarr store can avoid the toil of downloading/organizing/loading/preprocessing data for many users (in this case all of us). 

We are using [arco-era5](https://github.com/google-research/arco-era5) dataset. This is a great example of public ARCO data, and while it represents a large amount of work (ERA5 is notoriously difficult to download and process), I hope this will convince you of the downstream benefits of work like this and hopefully encourage you to [ingest]() other relevant datasets into the LEAP-Data Library. 

> If everyone does this for their favorite dataset, eventually all data loading becomes as easy as a copy and paste 😜

In [ ]:
# Literally copied and pasted this from the repos README
import xarray as xr

ar_full_37_1h = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2/'
)
ar_full_37_1h

In [ ]:
# how big is this whole thing?
print(f"The full dataset is {ar_full_37_1h.nbytes/1e12}TB large")

Ok so it only took us a few seconds to load half a PB? How is that possible?

We are not actually loading the data into memory, instead we are lazily loading the dataset (only loading the dimensions into memory) and get an xarray dataset backed by dask arrays.

## Exploration
The lazy loading enables us to load 'just what we need' for interactive exploration

In [ ]:
# load a single time slice and plot it
...

Nice, but ERA5 is pretty high res, and we might be interested in a certain region...

In [ ]:
# bonus: make this plot interactive
import hvplot.xarray
ar_full_37_1h['2m_temperature'].isel(time=200).hvplot(rasterize=True)

## Perform an analysis and save the aggregated results

This flexibility lets you easily build up analysis steps, test them out on small subsets or slices of the data, and refine the methods. 

After these initial stages you might want to save a derived data produce, especially if the product is relatively small, and the computation takes a while (more on how to scale the compute in a different tutorial).

For the sake of this tutorial, lets create a histogram of the surface temperature over NY state for each timestep.

> use 

In [ ]:
# bounding box [lon0, lon1, lat0, lat1]
ny_bbox = [
    360-79.762152,
    360-71.856214,
    45.01585,
    40.496103
] # latitudes are not monotonically increasing. Maybe we should raise an issue?

# select the 2m_temperature for NY state and plot a single time slice
...

In [ ]:
from xhistogram.xarray import histogram
import numpy as np

# use xhistogram to create a time resolved histogram array 
# (select only 5000 time elements to keep computation shorter at the end)
...

In [ ]:
# Plot the temperature histogram for the first and last time step
...

In [ ]:
# lets store this output as a zarr store
out_path = f"gs://leap-scratch/{os.environ['JUPYTERHUB_USER']}/annual_meeting_demo/era5/temp_hist.zarr"

# stream the computation results directly to a cloud zarr store!
...


## Reusing the output

So now you can hand of this store to all your colleagues and everyone can explore the hourly histograms without having to run this long computation.

In [ ]:
# reload the zarr store into an xarray dataset, and load it into memory (hint: `chunks=None`)

temp_hist_reloaded = xr.open_dataset(
    ...
)
temp_hist_reloaded

In [ ]:
# plot a 2d plot of histogram bins vs time
...